# Proporciones altas de fondos en test - usando HOG

## Imports

In [1]:
# Encadenar iterables
from itertools import chain

# Proporciona una barra de progreso rápida
from tqdm import tqdm

# Interfaz para hacer gráficos y visualizaciones
import matplotlib.pyplot as plt

# Computación científica
import numpy as np

# Manipulación de datos
import pandas as pd

# Extraer parches (pequeños subconjuntos de imágenes) de imágenes
from sklearn.feature_extraction.image import PatchExtractor

# data: conjunto de datos de muestra y funciones de carga
# color: convertir imágenes entre espacios de color
# feature: funciones para identificar y extraer características de imágenes
from skimage import data, color, feature

# Cambiar el tamaño de una imagen
from skimage.transform import resize, rescale

# Descarga y carga en memoria un conjunto de datos de imágenes de caras de personas famosas
from sklearn.datasets import fetch_lfw_people

# Modelos
from sklearn.linear_model import LogisticRegression

# Train test split
from sklearn.model_selection import train_test_split

# Matriz de confusión
from sklearn.metrics import confusion_matrix

# La curva ROC
from sklearn.metrics import roc_curve

# Classification report
from sklearn.metrics import classification_report

# Métricas varias
from sklearn.metrics import accuracy_score, balanced_accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

## Funciones auxiliares

In [2]:
# Función para extraer porciones de una imagen
def extract_patches(img, N, scale=1.0, patch_size=(62,47), random_state=0):
    # Calcula el tamaño del parche extraído basado en el factor de escala dado
    H = img.shape[0]
    W = img.shape[1]
    H_patch = min(H , int(scale * patch_size[0]))
    W_patch = min(W , int(scale * patch_size[1]))
    extracted_patch_size = (H_patch, W_patch)

    # Inicializa un objeto PatchExtractor con el tamaño de parche calculado,
    # el número máximo de parches, y una semilla de estado aleatorio
    extractor = PatchExtractor(patch_size=extracted_patch_size, max_patches=N, random_state=random_state)

    # Extrae parches de la imagen dada
    # img[np.newaxis] se utiliza la entrada de PatchExtractor es un conjunto de imágenes
    patches = extractor.transform(img[np.newaxis])

    # Si el factor de escala no es 1, redimensiona cada parche extraído
    # al tamaño del parche original
    if scale != 1:
        patches = np.array([resize(patch, patch_size) for patch in patches])

    # Devuelve la lista de parches extraídos (y posiblemente redimensionados)
    return patches

In [3]:
def non_max_suppression(indices, Ni, Nj, overlapThresh):
    # Si no hay rectángulos, regresar una lista vacía
    if len(indices) == 0:
        return []

    # Si las cajas son enteros, convertir a flotantes
    if indices.dtype.kind == "i":
        indices = indices.astype("float")

    # Inicializar la lista de índices seleccionados
    pick = []

    # Tomar las coordenadas de los cuadros
    x1 = np.array([indices[i,0] for i in range(indices.shape[0])])
    y1 = np.array([indices[i,1] for i in range(indices.shape[0])])
    x2 = np.array([indices[i,0]+Ni for i in range(indices.shape[0])])
    y2 = np.array([indices[i,1]+Nj for i in range(indices.shape[0])])

    # Calcula el área de los cuadros y ordena los cuadros
    area = (x2 - x1 + 1) * (y2 - y1 + 1)
    idxs = np.argsort(y2)

    # Mientras todavía hay índices en la lista de índices
    while len(idxs) > 0:
        # Toma el último índice de la lista y agrega el índice a la lista de seleccionados
        last = len(idxs) - 1
        i = idxs[last]
        pick.append(i)

        # Encontrar las coordenadas (x, y) más grandes para el inicio de la caja y las coordenadas (x, y) más pequeñas para el final de la caja
        xx1 = np.maximum(x1[i], x1[idxs[:last]])
        yy1 = np.maximum(y1[i], y1[idxs[:last]])
        xx2 = np.minimum(x2[i], x2[idxs[:last]])
        yy2 = np.minimum(y2[i], y2[idxs[:last]])

        # Calcula el ancho y alto de la caja
        w = np.maximum(0, xx2 - xx1 + 1)
        h = np.maximum(0, yy2 - yy1 + 1)

        # Calcula la proporción de superposición
        overlap = (w * h) / area[idxs[:last]]

        # Elimina todos los índices del índice de lista que tienen una proporción de superposición mayor que el umbral proporcionado
        idxs = np.delete(idxs, np.concatenate(([last], np.where(overlap > overlapThresh)[0])))

    # Devuelve solo las cajas seleccionadas
    return indices[pick].astype("int")

In [4]:
# Define una función para realizar una ventana deslizante (sliding window) sobre una imagen.
def sliding_window(img,
                   patch_size=(62,47),  # Define el tamaño del parche (patch) basado en el primer parche positivo por defecto
                   istep=2,  # Paso de desplazamiento en la dirección i (verticalmente)
                   jstep=2,  # Paso de desplazamiento en la dirección j (horizontalmente)
                   scale=1.0):  # Factor de escala para ajustar el tamaño del parche

    # Calcula las dimensiones Ni y Nj del parche ajustadas por el factor de escala.
    Ni, Nj = (int(scale * s) for s in patch_size)

    # Itera a lo largo de la imagen en la dirección i
    for i in range(0, img.shape[0] - Ni, istep):
        # Itera a lo largo de la imagen en la dirección j
        for j in range(0, img.shape[1] - Ni, jstep):

            # Extrae el parche de la imagen usando las coordenadas actuales i, j.
            patch = img[i:i + Ni, j:j + Nj]

            # Si el factor de escala es diferente de 1, redimensiona el parche al tamaño original del parche.
            if scale != 1:
                patch = resize(patch, patch_size)

            # Usa yield para devolver las coordenadas actuales y el parche.
            # Esto convierte la función en un generador.
            yield (i, j), patch

In [5]:
# Función que devuelve el número de detecciones brutas y procesadas para diversas escalas
# Esta función asume conocidos model, size y los parámetros de las HOG
def detections_by_scale(test_image, test_scales, step, thresholds=[0.5]):
    raw_detections = []
    detections = []

    for scale in tqdm(test_scales):
        raw_detections_scale = []
        detections_scale = []

        # Ventana deslizante
        indices, patches = zip(*sliding_window(test_image, scale=scale, istep=step, jstep=step))

        # Calcula las características HOG para cada parche y las almacena en un array.
        patches_hog = np.array([feature.hog(patch,
                                            orientations=orientations,
                                            pixels_per_cell=pixels_per_cell,
                                            cells_per_block=cells_per_block) for patch in patches])
        # Predicción
        for thr in thresholds:
            labels = (model.predict_proba(patches_hog)[:,1]>=thr).astype(int)
            raw_detections_scale.append(labels.sum())
            Ni, Nj = (int(scale*s) for s in size)
            indices = np.array(indices)
            detecciones = indices[labels == 1]
            detecciones = non_max_suppression(np.array(detecciones),Ni,Nj, 0.3)
            detections_scale.append(len(detecciones))

        # Actualizamos las listas
        raw_detections.append(raw_detections_scale)
        detections.append(detections_scale)

    return np.array(raw_detections), np.array(detections)

In [6]:
# True Positive Rate
def tpr_scorer(clf, X, y):
  y_pred = clf.predict(X)
  cm = confusion_matrix(y, y_pred)
  tpr = cm[1,1]/(cm[1,1]+cm[1,0])
  return tpr

# False Positive Rate
def fpr_scorer(clf, X, y):
  y_pred = clf.predict(X)
  cm = confusion_matrix(y, y_pred)
  fpr = cm[0,1]/(cm[0,0]+cm[0,1])
  return fpr

# True Negative Rate
def tnr_scorer(clf, X, y):
  y_pred = clf.predict(X)
  cm = confusion_matrix(y, y_pred)
  tnr = cm[0,0]/(cm[0,0]+cm[0,1])
  return tnr

# True Negative Rate
def fnr_scorer(clf, X, y):
  y_pred = clf.predict(X)
  cm = confusion_matrix(y, y_pred,)
  fnr = cm[1,0]/(cm[1,0]+cm[1,1])
  return fnr


## Dataset de rostros (LFW)

In [7]:
# Cargamos el dataset
faces = fetch_lfw_people()
positive_patches = faces.images
positive_patches.shape

(13233, 62, 47)

In [8]:
# Dividimos en train y test
positive_patches_train, positive_patches_test = train_test_split(
    positive_patches,
    test_size=0.1,
    random_state=42
)

In [9]:
print('Shape train: ',positive_patches_train.shape)
print('Shape test: ',positive_patches_test.shape)

Shape train:  (11909, 62, 47)
Shape test:  (1324, 62, 47)


## Dataset de fondos

In [10]:
# Tomamos algunas imágenes de sklearn
imgs = ['camera',
        'text',
        'coins',
        'moon',
        'page',
        'clock',
        'immunohistochemistry',
        'chelsea',
        'coffee',
        'hubble_deep_field'
        ]

backgrounds = []
for name in imgs:
    img = getattr(data, name)()
    if len(img.shape) == 3 and img.shape[2] == 3:  # Chequeamos si la imagen es RGB
        img = color.rgb2gray(img)
    backgrounds.append(img)

# Imagenes caseras adicionales
for i in range(31):
    filename = str(i)+'.jpg'
    img = plt.imread(filename)
    img = color.rgb2gray(img)
    backgrounds.append(img)

print(len(backgrounds))

41


## Definimos el modelo, los datos y las HOG

In [11]:
# Modelo
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier

model = RandomForestClassifier(n_estimators=100, max_depth = 5)

# Resolución de los rostros
resolution = 1

# Fondos
scales = [0.5,1,2,4,8]
proportion_train = 10
proportion_test = 100
num_patches_train = int((proportion_train * len(positive_patches_train))/(len(scales) * len(backgrounds)))
num_patches_test = int((proportion_test * len(positive_patches_test))/(len(scales) * len(backgrounds)))

# HOG
orientations = 9
pixels_per_cell = (8, 8)
cells_per_block = (3, 3)

# Nombre del experimento
model_name = str(model)
experiment_name = model_name
experiment_name += '_R_' + str(resolution)
experiment_name += '_S_' + str(scales)
experiment_name += '_PTrain_' + str(proportion_train)
experiment_name += '_PTest_' + str(proportion_test)
experiment_name += '_O_' + str(orientations)
experiment_name += '_C_' + str(pixels_per_cell)
experiment_name += '_B_' + str(cells_per_block)

print(experiment_name)

RandomForestClassifier(max_depth=5)_R_1_S_[0.5, 1, 2, 4, 8]_PTrain_10_PTest_100_O_9_C_(8, 8)_B_(3, 3)


In [12]:
# Tamaño de las imágenes de rostros

# Train
positive_patches_train = np.array(
    [rescale(positive_patches_train[i], resolution)
    for i in tqdm(range(len(positive_patches_train)))]
    )

# Test
positive_patches_test = np.array(
    [rescale(positive_patches_test[i], resolution)
    for i in tqdm(range(len(positive_patches_test)))]
    )

size = positive_patches_train[0].shape
print('Tamaño de los rostros: ',size)

100%|██████████| 1324/1324 [00:01<00:00, 961.49it/s] 

Tamaño de los rostros:  (62, 47)


In [ ]:
# Extraemos las imágenes de fondo

# Train
negative_patches_train = np.vstack(
    [extract_patches(im, num_patches_train, scale, random_state=42)
    for im in tqdm(backgrounds, desc='Procesando imágenes train')
    for scale in scales]
    )

# Test
negative_patches_test = np.vstack(
    [extract_patches(im, num_patches_test, scale, random_state=0)
    for im in tqdm(backgrounds, desc='Procesando imágenes test')
    for scale in scales]
    )

print('Shape train: ',negative_patches_train.shape)
print('Shape test: ',negative_patches_test.shape)

Procesando imágenes train:   2%|▏         | 1/41 [00:16<11:07, 16.69s/it]

In [ ]:
# Armamos la matriz de features y el vector de etiquetas

# Train
X_train = np.array(
    [feature.hog(image=im,
                 orientations=orientations,
                 pixels_per_cell=pixels_per_cell,
                 cells_per_block=cells_per_block)
    for im in tqdm(chain(positive_patches_train, negative_patches_train))]
    )
y_train = np.zeros(X_train.shape[0])
y_train[:positive_patches_train.shape[0]] = 1

# Test
X_test = np.array(
    [feature.hog(image=im,
                 orientations=orientations,
                 pixels_per_cell=pixels_per_cell,
                 cells_per_block=cells_per_block)
    for im in tqdm(chain(positive_patches_test, negative_patches_test))]
    )
y_test = np.zeros(X_test.shape[0])
y_test[:positive_patches_test.shape[0]] = 1

In [ ]:
print('Shape X_train: ', X_train.shape)
print('Shape y_train: ', y_train.shape)
print('Shape X_test: ', X_test.shape)
print('Shape y_test: ', y_test.shape)

## Entrenamiento y evaluación del modelo

In [ ]:
# Entrenamos el modelo
model.fit(X_train,y_train)

In [ ]:
# Predicciones
y_pred = model.predict(X_test)
y_pred_proba = model.predict_proba(X_test)[:,1]

In [ ]:
# Métrcias
acc = accuracy_score(y_test, y_pred)
b_acc = balanced_accuracy_score(y_test, y_pred)
prec = precision_score(y_test,y_pred,average='macro')
rec = recall_score(y_test, y_pred, average='macro')
f1 = f1_score(y_test, y_pred)
auc = roc_auc_score(y_test, y_pred_proba)
tpr = tpr_scorer(model, X_test, y_test)
fpr = fpr_scorer(model, X_test, y_test)
tnr = tnr_scorer(model, X_test, y_test)
fnr = fnr_scorer(model, X_test, y_test)

# Guardamos en un dataframe
results = pd.Series(
    data={
        'ACCURACY': acc,
        'PRECISION': prec,
        'RECALL': rec,
        'F1': f1,
        'B_ACCURACY': b_acc,
        'AUC': auc,
        'TPR': tpr,
        'FPR': fpr,
        'TNR': tnr,
        'FNR': fnr
    }
)

print(results)

In [ ]:
print(classification_report(y_test,y_pred, digits=4))

In [ ]:
# Curva ROC y umbral óptimo
fig, ax = plt.subplots(1,2,figsize=(8, 8))

fpr, tpr, thresholds = roc_curve(y_test, y_pred_proba)
gmean = np.sqrt(tpr * (1 - fpr))
index = np.argmax(gmean)
thresholdOpt = round(thresholds[index], ndigits = 4)
fprOpt = round(fpr[index], ndigits = 4)
tprOpt = round(tpr[index], ndigits = 4)

ax[0].step(
    fpr,
    tpr,
    lw=1,
    alpha=1,
)

ax[0].plot(
    fprOpt,
    tprOpt,
    marker = 'o'
)

ax[0].set(
    xlim=[-0.05, 1.05],
    ylim=[-0.05, 1.05],
    xlabel="False Positive Rate",
    ylabel="True Positive Rate",
    title=f"Curva ROC",
)
ax[0].axis("square")

ax[1].set_aspect('equal')
ax[1].set_xlim([-0.05, 0.1])
ax[1].set_xbound(lower=-0.05, upper=0.1)
ax[1].set_ylim([0.85,1])
ax[1].set_ybound(lower=0.85, upper=1.0)

ax[1].step(
    fpr,
    tpr,
    lw=1,
    alpha=1,
)

ax[1].plot(
    fprOpt,
    tprOpt,
    marker = 'o'
)

ax[1].set(
    xlabel="False Positive Rate",
    ylabel="True Positive Rate",
    title=f"Zoom",
)

plt.tight_layout()
plt.show()

print(f'Umbral óptimo: {thresholdOpt}')
print(f'FPR: {fprOpt}, TPR: {tprOpt}')

In [ ]:
# Otra forma de calcular un umbral adecuado
indx = np.argmax(tpr>=0.90)
thresholdAde = thresholds[indx]
print('Umbral adecuado: ', thresholdAde)

## Test en la imagen del astronauta

In [ ]:
# Imagen de prueba
test_image = data.astronaut()
test_image = color.rgb2gray(test_image)
test_image = rescale(test_image, 0.5)
test_image = test_image[:160, 40:180]

In [ ]:
# Visualizamos la imagen
# Buscamos la escala de los rostros
fig, ax = plt.subplots()
ax.imshow(test_image, cmap='gray')

true_scale = 1
Ni, Nj = (int(true_scale * s) for s in size)

ax.add_patch(plt.Rectangle((0, 0), Nj, Ni, edgecolor='red', alpha=1, lw=1, facecolor='none'))
plt.show()

In [ ]:
# Utiliza la función de ventana deslizante en una imagen de prueba.
# zip(*...) toma las tuplas generadas y las descompone en índices y parches.
indices, patches = zip(*sliding_window(test_image, scale=1))

# Calcula las características HOG para cada parche y las almacena en un array.
patches_hog = np.array([feature.hog(patch,
                                    orientations=orientations,
                                    pixels_per_cell=pixels_per_cell,
                                    cells_per_block=cells_per_block) for patch in patches])

# Muestra la forma del array de características HOG.
patches_hog.shape

### Desempeño según umbrales

In [ ]:
# Predicción

# Umbral default
labels_default = model.predict(patches_hog).astype(int)

# Umbral óptimo
labels_optimo = (model.predict_proba(patches_hog)[:,1]>=thresholdOpt).astype(int)

# Umbral adecuado
labels_adecuado = (model.predict_proba(patches_hog)[:,1]>=thresholdAde).astype(int)

In [ ]:
Ni, Nj = (int(true_scale*s) for s in size)
indices = np.array(indices)

# Umbral default
detecciones_default = indices[labels_default == 1]
detecciones_default = non_max_suppression(np.array(detecciones_default),Ni,Nj, 0.3)

# Umbral optimo
detecciones_optimo = indices[labels_optimo == 1]
detecciones_optimo = non_max_suppression(np.array(detecciones_optimo),Ni,Nj, 0.3)

# Umbral adecuado
detecciones_adecuado = indices[labels_adecuado == 1]
detecciones_adecuado = non_max_suppression(np.array(detecciones_adecuado),Ni,Nj, 0.3)

In [ ]:
# Visualizamos las detecciones
fig, ax = plt.subplots(1,3, figsize=(8,4))

# Umbral default
ax[0].imshow(test_image, cmap='gray')
ax[0].axis('off')

for i, j in detecciones_default:
    ax[0].add_patch(plt.Rectangle((j, i), Nj, Ni, edgecolor='red',alpha=1, lw=1, facecolor='none'))

ax[0].set_title('Default')

# Umbral óptimo
ax[1].imshow(test_image, cmap='gray')
ax[1].axis('off')

for i, j in detecciones_optimo:
    ax[1].add_patch(plt.Rectangle((j, i), Nj, Ni, edgecolor='red',alpha=1, lw=1, facecolor='none'))

ax[1].set_title('Óptimo')

# Umbral adecuado
ax[2].imshow(test_image, cmap='gray')
ax[2].axis('off')

for i, j in detecciones_adecuado:
    ax[2].add_patch(plt.Rectangle((j, i), Nj, Ni, edgecolor='red',alpha=1, lw=1, facecolor='none'))

ax[2].set_title('Adecuado')

plt.tight_layout()
plt.show()

### Desempeño en varias escalas según umbral

In [ ]:
# Escalas a testear
test_scales = np.linspace(0.125, 2, 50)

In [ ]:
raw_detections, detections = detections_by_scale(
    test_image,
    test_scales,
    step=2,
    thresholds=[0.5, thresholdOpt, thresholdAde]
    )

In [ ]:
number_faces = 1

fig, ax = plt.subplots(1,2, figsize=(12,4))

ax[0].set_title('Bruto')
ax[0].axvline(x=true_scale, ls = '--', color='red')
ax[0].step(test_scales, raw_detections[:,0], label = 'Default')
ax[0].step(test_scales, raw_detections[:,1], label = 'Óptimo')
ax[0].step(test_scales, raw_detections[:,2], label = 'Adecuado')
ax[0].grid(True)
ax[0].set_xlabel('Escalas')
ax[0].set_ylabel('Detecciones')
ax[0].legend()

ax[1].set_title('Procesado')
ax[1].axvline(x=true_scale, ls = '--', color='red')
ax[1].axhline(y=number_faces, ls = '--', color='red')
ax[1].step(test_scales, detections[:,0], label = 'Default')
ax[1].step(test_scales, detections[:,1], label = 'Óptimo')
ax[1].step(test_scales, detections[:,2], label = 'Adecuado')
ax[1].grid(True)
ax[1].set_xlabel('Escalas')
ax[1].set_ylabel('Detecciones')
ax[1].legend()
plt.show()